In [1]:
# Social Reciprocity Detection Simulation
import cv2
import mediapipe as mp
import time

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

cue_given = False
cue_start_time = None
response_time = None
joint_attention_attempted = False
timeout_sec = 4

cap = cv2.VideoCapture(0)
print("Simulate a 'name-call' using a wave gesture.")
print("Watch if the subject turns head or points.")
print("Press 'q' to quit.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Get landmarks
        nose = results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]
        left_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
        left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]

        # Simulated wave cue (wrist above shoulder)
        if left_wrist.y < left_shoulder.y and not cue_given:
            cue_given = True
            cue_start_time = time.time()
            print("Gesture cue given - checking for social response...")

        # Detect head turn (nose x deviation)
        if cue_given:
            if nose.x < 0.4 or nose.x > 0.6:
                response_time = time.time() - cue_start_time
                cue_given = False
                if response_time < timeout_sec:
                    cv2.putText(frame, "✅ Social Response Detected", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                else:
                    cv2.putText(frame, "⚠️ Delayed Social Response", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Check if hand is pointing (arm extended horizontally)
        left_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW]
        if abs(left_wrist.y - left_elbow.y) < 0.1 and abs(left_elbow.y - left_shoulder.y) < 0.1:
            joint_attention_attempted = True
            cv2.putText(frame, "🫵 Joint Attention Attempt", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 215, 0), 2)

    # Display frame
    cv2.imshow("Social Reciprocity Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'mediapipe'